In [4]:
%%time
# Import the relevant modules to be used later
from __future__ import print_function
import gzip
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import struct
import sys

try: 
    from urllib.request import urlretrieve 
except ImportError: 
    from urllib import urlretrieve

# Config matplotlib for inline plotting
%matplotlib inline

Wall time: 291 ms


## Data Augmentation

In [12]:
from PIL import Image, ImageOps

In [22]:
%%time
import csv
count = 0
with open('data/train_labels.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if count > 0:
            name = row[0]
            img = Image.open('data/train/' + name + '.png')
            tmp = img.rotate(90)
            tmp.save('data/train_90/' + name + '.png')
            tmp = img.rotate(180)
            tmp.save('data/train_180/' + name + '.png')
            tmp = img.rotate(270)
            tmp.save('data/train_270/' + name + '.png')
            tmp = ImageOps.flip(img)
            tmp.save('data/train_flip/' + name + '.png')
            tmp = ImageOps.mirror(img)
            tmp.save('data/train_mirror/' + name + '.png')
        count = count + 1


Wall time: 1min 28s


In [42]:
def orient(input, rotate):
    new_ori = int(input) + int((rotate / 90))
    if new_ori > 3:
        new_ori = new_ori - 4
    if new_ori < 0:
        new_ori = new_ori + 4
    return new_ori

In [40]:
print(orient(3,90))
print(orient(3,180))
print(orient(3,270))

t = 3
o = t
if (t == 1) or (t == 3):
    o = orient(t,180)
print(o)

0
1
2
1


In [49]:
%%time
import csv
count = 0
name = ''
ori = 0
larray = ['1 0 0 0', '0 1 0 0', '0 0 1 0', '0 0 0 1']
label = ''
feature = ''
output_train = []
output_train90 = []
output_train180 = []
output_train270 = []
output_test = []
with open('data/train_labels.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if count > 0:
            name = row[0]
            ori = int(row[1]) - 1
            label = larray[ori]
            feature = loadImage('data/train/' + name + '.png')
            output_train.append('|labels {} |features {}\n'.format(label, feature))
            label = larray[orient(ori,90)]
            feature = loadImage('data/train_90/' + name + '.png')
            output_train90.append('|labels {} |features {}\n'.format(label, feature))
            label = larray[orient(ori,180)]
            feature = loadImage('data/train_180/' + name + '.png')
            output_train180.append('|labels {} |features {}\n'.format(label, feature))
            label = larray[orient(ori,270)]
            feature = loadImage('data/train_270/' + name + '.png')
            output_train270.append('|labels {} |features {}\n'.format(label, feature))
            t = ori
            if (t == 1) or (t == 3):
                ori = orient(t,180)
            label = larray[ori]
            feature = loadImage('data/train_mirror/' + name + '.png')
            output_test.append('|labels {} |features {}\n'.format(label, feature))
        count += 1


import time
import datetime
ts = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
file_train = 'data/Train-64x64x80000_' + ts + '.txt'
file_test = 'data/Test-64x64x20000_' + ts + '.txt'
print("Creating: ", file_train )
with open(file_train, 'w') as f:
    for row in output_train:
        f.write(row)
    for row in output_train90:
        f.write(row)
    for row in output_train180:
        f.write(row)
    for row in output_train270:
        f.write(row)
print("Done.")

print("Creating: ", file_test )
with open(file_test, 'w') as f:
    for row in output_test:
        f.write(row)
    print("Done.")


Creating:  data/Train-64x64x80000_20180718_095800.txt
Done.
Creating:  data/Test-64x64x20000_20180718_095800.txt
Done.
Wall time: 8min 22s


## Training data prep

In [2]:
import cv2
from PIL import Image

def loadImage2(filename):
    img = Image.open(filename)
    pixels = img.load()
    img_data = ''
    for i in range(img.size[0]):    
        for j in range(img.size[1]): 
            img_data = img_data + ' ' + str(pixels[i,j])
    return img_data

def loadImage(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img_data = ' '.join(str(x) for x in img.flatten().tolist())
    return img_data


from keras.preprocessing.image import img_to_array 
from sklearn.preprocessing import LabelBinarizer 
from imutils import paths 
import random 
import cv2 
import os
import numpy as np

imagePaths = sorted(list(paths.list_images('C:/Users/dachou/notebooks/DAT264x/data/test-1'))) 

image = cv2.imread('C:/Users/dachou/notebooks/DAT264x/data/test-1/200000.png', cv2.COLOR_RGB2GRAY) 
print(image.shape)
print(image.dtype)
print(image.size)
gray_img = cv2.imread('C:/Users/dachou/notebooks/DAT264x/data/test-1/200000.png', cv2.IMREAD_GRAYSCALE)  # grayscale
print(gray_img.shape)
print(gray_img.dtype)
print(gray_img.size)


#img = loadImage('C:/Users/dachou/notebooks/DAT264x/data/test-1/200000.png')
flat = gray_img.flatten()
reshape = flat.reshape(64, 64)
cv2string = ' '.join(str(x) for x in flat.tolist())
pilstring = loadImage2('data/test-1/200000.png')
print(pilstring)


In [3]:
%%time
#if os.path.isfile("data/train_labels.csv")
import csv
count = 0
larray = ['1 0 0 0', '0 1 0 0', '0 0 1 0', '0 0 0 1']
label = ''
feature = ''
output = []
with open('data/train_labels.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if count > 0:
            label = larray[int(row[1])]
            feature = loadImage('C:/Users/dachou/notebooks/DAT264x/data/train/' + row[0] + '.png')
            output.append('|labels {} |features {}\n'.format(label, feature))
            #print('({0}) file: {1} label: {2}'.format(count, row[0], row[1]) + ' onehot: ' + label)
        count += 1


Wall time: 32.3 s


In [4]:
#Save file
import time
import datetime
filename = 'data/Data-20000_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S') + '.txt'
if not os.path.isfile(filename):
    print("Saving", filename )
    with open(filename, 'w') as f:
        for row in output:
            f.write(row)
    print("File save complete.")
else:
    print("File already exists", filename)

Saving data/Data-submit-20000_20180720_092205.txt
File save complete.


# Test data

In [88]:
%%time
test_dir = 'data/test'
test_files = os.listdir(test_dir)
label = '0 0 0 0'
feature = ''
output = []
print("Reading files...")
for file in test_files:
    feature = loadImage(test_dir + '/' + file)
    output.append('|labels {} |features {}\n'.format(label, feature))

import time
import datetime
filename = 'data/Test-10_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S') + '.txt'
if not os.path.isfile(filename):
    print("Saving", filename )
    with open(filename, 'w') as f:
        for row in output:
            f.write(row)
    print("File save complete.")
else:
    print("File already exists", filename)

Reading files...
Saving data/Test-10_20180720_070801.txt
File save complete.
Wall time: 31 s
